In [6]:
!git clone https://github.com/justinpinkney/stable-diffusion.git
%cd stable-diffusion
!pip install --upgrade pip
!pip install -r requirements.txt

!pip install --upgrade keras # on lambda stack we need to upgrade keras
!pip uninstall -y torchtext # on colab we need to remove torchtext

fatal: destination path 'stable-diffusion' already exists and is not an empty directory.
/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/stable-diffusion/stable-diffusion
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r requirements.txt (line 23))
ERROR: Exception:
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pip/_internal/vcs/git.py", line 368, in get_remote_url
    found_remote = remotes[0]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "/an

In [7]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [8]:
import os
from PIL import Image, UnidentifiedImageError

def preprocess_images(input_dir, output_dir, size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    
    for file_name in os.listdir(input_dir):  # Iterate over files in the input directory
        img_path = os.path.join(input_dir, file_name)
        
        # Ensure the file has a .jpg extension
        if not file_name.lower().endswith(".jpg"):
            print(f"Skipping non-JPG file: {file_name}")
            continue
        
        try:
            # Open and process the image
            with Image.open(img_path) as img:
                img = img.convert("RGB")  # Ensure RGB format
                img = img.resize(size)    # Resize the image
                img.save(os.path.join(output_dir, file_name))  # Save the preprocessed image
        except UnidentifiedImageError:
            print(f"Skipping corrupted image file: {file_name}")
        except PermissionError:
            print(f"Skipping file due to permission error: {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Use the correct dataset path
input_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/images"
output_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/preprocessed_images"

print("Input Path Exists:", os.path.exists(input_path))
print("Output Path Exists:", os.path.exists(output_path))

# Run the preprocessing function
preprocess_images(input_path, output_path)


Input Path Exists: True
Output Path Exists: True
Skipping non-JPG file: .amlignore
Skipping non-JPG file: .amlignore.amltmp


In [9]:
pip install datasets


Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from PIL import Image, UnidentifiedImageError
from datasets import Dataset
import numpy as np
from torch.utils.data import DataLoader

# Preprocess images: resize and save to output directory
def preprocess_images(input_dir, output_dir, size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)
    valid_extensions = (".jpg", ".jpeg", ".png")
    for file_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, file_name)
        if not file_name.lower().endswith(valid_extensions):  # Skip non-image files
            print(f"Skipping non-image file: {file_name}")
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            img = img.resize(size)
            img.save(os.path.join(output_dir, file_name))
        except UnidentifiedImageError:
            print(f"Skipping corrupted file: {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Define paths
input_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/images"
output_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/preprocessed_images"
captions_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/captions.txt"

# Preprocess images
preprocess_images(input_path, output_path)

# Ensure captions.txt exists
assert os.path.exists(captions_path), "The captions.txt file is missing!"

# Load captions and prepare dataset
data = []
with open(captions_path, "r") as f:
    for line in f:
        image_name, caption = line.strip().split("   ")  # Split using double space
        image_path = os.path.join(output_path, image_name)
        if os.path.exists(image_path):
            data.append({"image": image_path, "text": caption})

# Convert to Hugging Face Dataset
data_dict = {
    "image": [item["image"] for item in data],
    "text": [item["text"] for item in data],
}
ds = Dataset.from_dict(data_dict)

# Define dynamic preprocessing function
def preprocess_data(batch):
    processed_images = []
    for img_path in batch["image"]:
        try:
            image = Image.open(img_path).convert("RGB")
            image = image.resize((256, 256))
            processed_images.append(np.array(image))
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            processed_images.append(np.zeros((256, 256, 3), dtype=np.uint8))  # Placeholder for errors
    batch["image"] = processed_images
    return batch

# Apply preprocessing dynamically
ds = ds.with_transform(preprocess_data)

# Debug: Verify image types and shapes
for i in range(3):
    print(f"Sample {i} image type: {type(ds[i]['image'])}, Shape: {ds[i]['image'].shape}")

# Define a simple collate function for DataLoader
def collate_fn(batch):
    images = np.stack([item["image"] for item in batch])  # Stack as NumPy arrays
    captions = [item["text"] for item in batch]  # Extract captions
    return {"images": images, "captions": captions}

# Create DataLoader
dataloader = DataLoader(ds, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Test the DataLoader
for batch in dataloader:
    print("Images shape:", batch["images"].shape)  # Should be (batch_size, 256, 256, 3)
    print("Captions:", batch["captions"])
    break


Skipping non-image file: .amlignore
Skipping non-image file: .amlignore.amltmp
Sample 0 image type: <class 'numpy.ndarray'>, Shape: (256, 256, 3)
Sample 1 image type: <class 'numpy.ndarray'>, Shape: (256, 256, 3)
Sample 2 image type: <class 'numpy.ndarray'>, Shape: (256, 256, 3)
Images shape: (8, 256, 256, 3)
Captions: ['A brunette female with soft beige cat ears blending seamlessly into her hair. She is wearing a tan off-shoulder top, with a neutral background that highlights her warm and natural appearance.', 'A blonde female with natural light-brown cat ears and expressive blue eyes, dressed in a light cardigan and tank top. She is standing on a city street with blurred figures and urban buildings in the background.', 'A blonde female with natural light-brown cat ears, expressive blue eyes, and a neutral-toned cardigan, standing in an urban environment with soft natural lighting.', 'A brunette female with natural beige-and-brown cat ears and soft green-brown eyes, dressed in a simpl

In [11]:
!pip install huggingface_hub

In [12]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [13]:
from huggingface_hub import notebook_login
notebook_login()

Token has not been saved to git credential helper.


In [14]:
!pip install diffusers transformers accelerate

In [15]:
!pip install --upgrade diffusers transformers accelerate datasets huggingface_hub xformers

import os
from PIL import Image
from datasets import Dataset
import numpy as np
from torch.utils.data import DataLoader

In [16]:
pip install diffusers


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install diffusers transformers accelerate torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [18]:
from diffusers import UNet2DConditionModel, AutoencoderKL, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from torch.optim import AdamW
from accelerate import Accelerator
import torch

2024-12-16 18:15:43.405501: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-16 18:15:46.210773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 18:15:47.063844: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 18:15:47.307295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 18:15:49.259851: I tensorflow/core/platform/cpu_feature_guar

In [27]:

# Step 1: Prepare Dataset
# Define paths
input_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/images"
image_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/preprocessed_images"
caption_file = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/my_dataset/captions.txt"

# Load captions
data = []
with open(caption_file, "r") as f:
    for line in f:
        image_name, caption = line.strip().split("   ")  # Assuming '   ' separates filename and caption
        image_path = os.path.join(image_dir, image_name)
        if os.path.exists(image_path):
            data.append({"image": image_path, "text": caption})

# Create Hugging Face Dataset
ds = Dataset.from_dict({
    "image": [item["image"] for item in data],
    "text": [item["text"] for item in data],
})

# Preprocess images
def preprocess_images(sample):
    image = Image.open(sample["image"]).convert("RGB")
    image = image.resize((512, 512))  # Resize to 512x512 for Stable Diffusion XL
    sample["image"] = np.array(image)
    return sample

ds = ds.map(preprocess_images, batched=False)






Map:   0%|          | 0/27 [00:00<?, ? examples/s]

In [36]:

# Step 2: Define Training Parameters
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
num_epochs = 3
train_batch_size = 4
learning_rate = 5e-6
gradient_accumulation_steps = 2
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Stable Diffusion XL Components
unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet").to(device)
vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to(device)
text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder").to(device)
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")
noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")


In [37]:
# Step 3: Prepare DataLoader
def collate_fn(batch):
    images = torch.tensor([item["image"] for item in batch]).permute(0, 3, 1, 2) / 255.0  # Normalize
    captions = [item["text"] for item in batch]
    return {"images": images.to(device), "captions": captions}

dataloader = DataLoader(ds, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)

# Step 4: Set Up Optimizer and Accelerator
optimizer = AdamW(unet.parameters(), lr=learning_rate)
accelerator = Accelerator(mixed_precision="fp16")
unet, optimizer, dataloader = accelerator.prepare(unet, optimizer, dataloader)


In [49]:
import torch
import torch.nn as nn

# Step 5: Training Loop
for epoch in range(num_epochs):
    unet.train()
    for step, batch in enumerate(dataloader):
        images = batch["images"].to(device)
        captions = batch["captions"]

        # Tokenize captions
        inputs = tokenizer(captions, padding="max_length", truncation=True, max_length=77, return_tensors="pt").to(device)
        encoder_hidden_states = text_encoder(inputs.input_ids)[0]  # Shape: (batch_size, seq_len, embedding_dim)

        # Add noise to images
        noise = torch.randn_like(images).to(device)  # Same shape as images
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (images.shape[0],), device=device).long()
        noisy_images = noise_scheduler.add_noise(images, noise, timesteps)

        # Use a time embedding layer to embed timesteps
        time_embedding_dim = encoder_hidden_states.shape[-1]  # This should match the embedding dimension of the text encoder
        time_embedder = nn.Embedding(noise_scheduler.config.num_train_timesteps, time_embedding_dim).to(device)

        # Lookup the time embeddings for the timesteps
        time_embeds = time_embedder(timesteps)  # Shape: (batch_size, embedding_dim)

        # Expand time_embeds to match the sequence length of encoder_hidden_states
        time_embeds = time_embeds.unsqueeze(1).expand(-1, encoder_hidden_states.shape[1], -1)  # Shape: (batch_size, seq_len, embedding_dim)

        # Concatenate text_embeds and time_embeds along the last dimension
        # This will create a tensor of shape (batch_size, seq_len, 2 * embedding_dim)
        combined_embeds = torch.cat([encoder_hidden_states, time_embeds], dim=-1)  # Shape: (batch_size, seq_len, 2 * embedding_dim)

        # Prepare additional conditioning kwargs
        added_cond_kwargs = {
            "text_embeds": combined_embeds,
            "time_ids": timesteps  # Add time_ids to conditioning kwargs
        }

        # Predict noise
        noise_pred = unet(noisy_images, timesteps, encoder_hidden_states, added_cond_kwargs=added_cond_kwargs).sample()  # Ensure the call is correct

        # Compute loss
        loss = torch.nn.functional.mse_loss(noise_pred, noise)
        accelerator.backward(loss)  # Using accelerator for backward pass
        
        # Optimizer step
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Log progress
        if step % 10 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")

# Step 6: Save Fine-Tuned Model
unet.save_pretrained("/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/model/stable-diffusion-xl-finetuned/unet")
text_encoder.save_pretrained("/mnt/batch/tasks/shared/LS_root/mounts/clusters/execinstance/code/Users/1155143473/Human Image Generation with Animal Features/model/stable-diffusion-xl-finetuned/text_encoder")


RuntimeError: Tensors must have same number of dimensions: got 3 and 2